In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys

In [2]:
## open csv file 
df = pd.read_csv('Project1.csv')
df.head()

,DeliveryID,Value,Duration,Start,End,Address
0,0,NaN,NaN,NaN,NaN,"4331 Overland Ave, Culver City, CA 90230"
1,1,30.0,30.0,10.0,12.0,"11106 1/2 National Blvd, Los Angeles. CA 90034"
2,2,80.0,45.0,12.0,2.0,"Culver City, 10797 Washington Blvd, Culver Cit..."
3,3,25.0,15.0,2.0,4.0,"10836 Washington Blvd, Culver City, CA 90232"
4,4,150.0,90.0,4.0,6.0,"3470 S Sepulveda Blvd, Los Angeles, CA 90034"


In [3]:
## Using Selenium to get the travel time
## time.sleep(2) allows the website to load for 2 seconds 

def getTime(start_addr,dest_addr):
    driver = webdriver.Chrome()
    driver.maximize_window()
    # get the link of google map
    driver.get('https://www.google.com/maps')
    elem = driver.find_element_by_xpath('//*[@id="searchbox-directions"]') # click on the direction
    elem.click()
    time.sleep(2)
    input_from = driver.find_element_by_xpath('//*[@id="sb_ifc51"]/input')
    input_from.send_keys(start_addr) # input starting point
    time.sleep(2)
    input_to= driver.find_element_by_xpath('//*[@id="sb_ifc52"]/input')
    input_to.send_keys(dest_addr) #input destination point
    input_to.send_keys(Keys.RETURN)
    time.sleep(2)
    # select car buttom
    car = driver.find_element_by_xpath('//*[@id="omnibox-directions"]/div/div[2]/div/div/div[1]/div[2]/button/div[1]')
    car.click()
    time.sleep(2)
    travel_time = driver.find_element_by_xpath('//*[@id="section-directions-trip-0"]/div[2]/div[1]/div[1]/div[1]/span[1]')
    text = travel_time.get_attribute('textContent')
    driver.close()
    time_value = text.split(' ') # split ['6 mins'] into ['6' , 'mins']
    
    return int(time_value[0])

In [4]:
destination = df.DeliveryID.tolist() # destination list

In [5]:
plan = [] # final delivery id plan
curr = 0 #current Delivery id
destination.remove(curr) # recomve the current location

In [6]:
## loop the funtion while destination list is not None

while destination:
    max_value = -9999;
    max_index = -9999;
    start_addr = df.Address[df['DeliveryID'] == curr].to_string(index = False)# get address based on the delivery id = curr
    for i in destination:
        print("start_addr "+start_addr)
        dest_addr = df.Address[df['DeliveryID'] == i].to_string(index = False)# get address based on the delivery id = destination
        print("dest_addr "+dest_addr)
        time_value = getTime(start_addr,dest_addr) # get travel time from above getTime function
        print("value 1: "+ str(time_value))
        Value = float(df.Value[df['DeliveryID'] == i].to_string(index = False))# get Value based on the delivery id = destination
        Duration = float(df.Duration[df['DeliveryID'] == i].to_string(index = False))# get Duration based on the delivery id = destination
        Profit = Value - (Value/Duration)*(Duration + time_value)# calculate the profit
        print("value 2: "+ str(Profit))
        if Profit > max_value:
            max_value = Profit
            max_index = i
    print("max "+str(max_index))
    plan.append(max_index)
    destination.remove(max_index) # remove point with max profit
    curr = max_index

start_addr  4331 Overland Ave, Culver City, CA 90230
dest_addr  11106 1/2 National Blvd, Los Angeles. CA 90034
value 1: 8
value 2: -8.0
start_addr  4331 Overland Ave, Culver City, CA 90230
dest_addr  Culver City, 10797 Washington Blvd, Culver Cit...
value 1: 3
value 2: -5.333333333333329
start_addr  4331 Overland Ave, Culver City, CA 90230
dest_addr  10836 Washington Blvd, Culver City, CA 90232
value 1: 3
value 2: -5.0
start_addr  4331 Overland Ave, Culver City, CA 90230
dest_addr  3470 S Sepulveda Blvd, Los Angeles, CA 90034
value 1: 7
value 2: -11.666666666666686
max 3
start_addr  10836 Washington Blvd, Culver City, CA 90232
dest_addr  11106 1/2 National Blvd, Los Angeles. CA 90034
value 1: 7
value 2: -7.0
start_addr  10836 Washington Blvd, Culver City, CA 90232
dest_addr  Culver City, 10797 Washington Blvd, Culver Cit...
value 1: 2
value 2: -3.555555555555557
start_addr  10836 Washington Blvd, Culver City, CA 90232
dest_addr  3470 S Sepulveda Blvd, Los Angeles, CA 90034
value 1: 6
v

In [7]:
# print plan as str

str_plan = 'The route is : '+str(plan[0]) +' '
for i in range(1,len(plan)):
    str_plan +=  '-> ' + str(plan[i])

print(str_plan)

The route is : 3 -> 2-> 1-> 4


In [8]:
# print plan in messagebox
# import tkinter
# import tkinter.messagebox

# Route = tkinter.messagebox.askokcancel("Route for delivery man",str_plan)

In [9]:
## prepare JSON for visualization

temp = pd.DataFrame()

temp['DeliveryID'] = plan

add = []
for i in plan:
    add.append(df.Address[df['DeliveryID'] == i].to_string(index = False))
    
temp['Address'] = add


temp.to_json('plan.json',orient='index')